# Referral Net Tracker - Data Gathering

This notebook explains the data gathering steps of the referral net tracker algorithm. The algorithm tracks referral funds in the Sky/Star Accessibility Rewards system.

## 1. Setup and Imports

First, we import the necessary libraries and set up our environment.

In [ ]:
import uuid
import datetime
import numpy as np
import json
import time
import pandas as pd
import os
from os.path import join, dirname
from dotenv import load_dotenv

# Set reward percentage and initial check date
reward_percentage = 0.4/100
checkFrom = datetime.datetime.fromisoformat('2030-12-20 23:59:59')

# Set up path and environment variables
base = os.path.abspath("")
import sys
## add your path to the code folder from your jupyter server root directry below.
# if you use local running ipython in vscode no path adaptions are neccessary
sys.path.append(os.path.join(base,'../sky_star_acessibility_rewards/code')) 

# Load environment variables
dotenv_path = join(dirname(os.path.dirname('__file__')), '.env')
load_dotenv(dotenv_path)

In [ ]:
os.path.join(base,'../sky_star_acessibility_rewards/code')

## 2. Database Connection Setup

We connect to both Snowflake and PostgreSQL databases to fetch and store data.

In [ ]:
import snowflake.connector
from sqlalchemy import create_engine

# Import configuration settings from config file
from common.config import (
    DATABASE_NAME,
    DATABASE_HOST,
    DATABASE_USER,
    DATABASE_PASSWORD,
    DATABASE_PORT,
    SNOWFLAKE_ACCOUNT,
    SNOWFLAKE_USER,
    SNOWFLAKE_PASSWORD,
    SNOWFLAKE_DATABASE
)

# Import custom modules for decoding
import common.stablelib.action_decoder.referral_fund_tracker_decode as rd

To execute the script you need access to a flipside snowflake equivalent, easiest to be found here: https://docs.flipsidecrypto.xyz/welcome-to-flipside/data/choose-your-flipside-plan/pro

In general the events table needs to have the following columns:
| Column | Description |
|--------|-------------|
| block_timestamp | When the event occurred |
| block_number | Block number of the event |
| tx_hash | Transaction hash |
| event_name | Type of event (e.g., transfer, deposit, withdraw) |
| dl | Decoded log data containing event-specific information |
| contract_address | Contract address |
| event_index | Order of event in transaction |

## 3. Database Connections

Establish connections to Snowflake and PostgreSQL databases.

In [ ]:
# Connect to Snowflake
snowflake_conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
    database=SNOWFLAKE_DATABASE,
)

# Connect to PostgreSQL
connection = "postgresql://"+DATABASE_USER+":"+DATABASE_PASSWORD+"@"+DATABASE_HOST+":"+DATABASE_PORT+"/"+DATABASE_NAME
engine = create_engine(connection) 

# Get sUSDS prices for later calculations
query_sUSDSprices="""SELECT DISTINCT ON (DATE_TRUNC('month', date)) token_id, date, price
FROM coingecko.market_data
where token_id = 'susds'
ORDER BY DATE_TRUNC('month', date) DESC, date DESC
"""
dfsUSDSPrices = pd.read_sql(query_sUSDSprices, engine)
dfsUSDSPrices['month'] = dfsUSDSPrices['date'].dt.to_period('M')
print('Got sUSDS prices')

## 4. Data Gathering - sUSDS Logs

First, we fetch logs from the sUSDS contract to track transfers and other events.

In [ ]:
# Start tracking time for performance measurement
all_timer = time.time()
timer = time.time()

print("Starting data gathering")
print("Date:", datetime.datetime.now())

# Query to fetch sUSDS logs
susds_logs = """
    SELECT * FROM ethereum.core.ez_decoded_event_logs WHERE contract_address = lower('0xa3931d71877C0E7a3148CB7Eb4463524FEc27fbD') AND tx_status = 'SUCCESS'
"""
dfSUsdsLogs = pd.read_sql(susds_logs, snowflake_conn)

# Process the data
dfSUsdsLogs.columns = dfSUsdsLogs.columns.str.lower()
dfSUsdsLogs['dl'] = dfSUsdsLogs['decoded_log'].apply(lambda x: json.loads(x))
dfSUsdsLogs = dfSUsdsLogs[[
    'block_timestamp', 'block_number', 'tx_hash',
    'event_name', 'dl', 'contract_address', 'contract_name', 'event_index', 'origin_function_signature', 'origin_from_address',
    'origin_to_address', 'topics'
]]

print(f"USDS data loaded: {time.time()-timer:.2f} seconds, {dfSUsdsLogs.shape[0]} rows")

## 5. Data Gathering - Cowswap Logs

Next, we fetch Cowswap-related events and create synthetic referrals based on deposit/stake events.

In [ ]:
timer = time.time()

# Query to fetch Cowswap logs
cowswap_logs_query = """
    SELECT *
    FROM ethereum.core.ez_decoded_event_logs AS del
    WHERE EXISTS (
        SELECT 1
        FROM ethereum.core.ez_decoded_event_logs AS del1
        WHERE del1.tx_hash = del.tx_hash
        AND del1.contract_address = LOWER('0x9008d19f58aabd9ed0d60971565aa8510560ab41')
    )
    AND EXISTS (
        SELECT 1
        FROM ethereum.core.ez_decoded_event_logs AS del2
        WHERE del2.tx_hash = del.tx_hash
        AND del2.contract_address IN (
            LOWER('0xa3931d71877C0E7a3148CB7Eb4463524FEc27fbD'),
            LOWER('0x0650CAF159C5A49f711e8169D4336ECB9b950275'),
            LOWER('0x10ab606B067C9C461d8893c47C7512472E19e2Ce')
        )
        AND del2.tx_status = 'SUCCESS'
    )
"""
cowswap_logs = pd.read_sql(cowswap_logs_query, snowflake_conn)
cowswap_logs.columns = cowswap_logs.columns.str.lower()

# Filter for deposit/staked events on relevant contracts
deposit_staked_df = cowswap_logs[(cowswap_logs['event_name'] == 'Deposit') | (cowswap_logs['event_name'] == 'Staked')]
deposit_staked_df = deposit_staked_df[deposit_staked_df['contract_address'].isin([
    '0xa3931d71877c0e7a3148cb7eb4463524fec27fbd',
    '0x0650caf159c5a49f711e8169d4336ecb9b950275',
    '0x10ab606b067c9c461d8893c47c7512472e19e2ce'
])]

# Create synthetic referrals for Cowswap (referral code 1003)
synthetic_referals = deposit_staked_df.copy()[0:0]
for index, row in deposit_staked_df.iterrows():
    if row['event_name'] == 'Deposit':
        referral = row.copy()
        tjson = json.loads(row['decoded_log'])
        tjson['referral'] = 1003  # Cowswap referral code
        referral['event_name'] = 'Referral'
        referral['decoded_log'] = json.dumps(tjson)
        synthetic_referals = synthetic_referals._append(referral)
    if row['event_name'] == 'Staked':
        staked = row.copy()
        amount = json.loads(row['decoded_log'])['amount']
        user = json.loads(row['decoded_log'])['user']
        staked['decoded_log'] = '{"amount": "' + amount + '", "referral": 1003, "user": "' + user + '"}'
        synthetic_referals = synthetic_referals._append(staked)

# Process the synthetic data
synthetic_referals['dl'] = synthetic_referals['decoded_log'].apply(lambda x: json.loads(x))
synthetic_referals = synthetic_referals[[
    'block_timestamp','block_number', 'tx_hash',
    'event_name', 'dl','contract_address', 'contract_name','event_index', 'origin_function_signature', 'origin_from_address',
    'origin_to_address', 'topics'
]]
synthetic_referals['event_name'] = synthetic_referals['event_name'].str.lower()

print(f"Cowswap data loaded: {time.time()-timer:.2f} seconds, {synthetic_referals.shape[0]} rows")

## 6. Data Gathering - LazySummer Logs

We also fetch LazySummer-related events and create synthetic referrals (referral code 1016).

In [ ]:
timer = time.time()

# Query to fetch LazySummer logs
lazysummer_logs_query = """
    SELECT * FROM ethereum.core.ez_decoded_event_logs
    WHERE tx_hash IN (
        SELECT d.tx_hash
        FROM ethereum.core.ez_decoded_event_logs d
        WHERE d.contract_address IN ('0xa3931d71877c0e7a3148cb7eb4463524fec27fbd', '0x0650caf159c5a49f711e8169d4336ecb9b950275', '0x10ab606b067c9c461d8893c47c7512472e19e2ce')
        AND d.event_name = 'Deposit'
        AND EXISTS (
            -- Check if the same transaction has a "Rebalanced" event (from any contract)
            SELECT 1
            FROM ethereum.core.ez_decoded_event_logs r
            WHERE r.tx_hash = d.tx_hash
            AND r.event_name = 'Rebalanced'
        )
    )
"""
lazysummer_logs_query = pd.read_sql(lazysummer_logs_query, snowflake_conn)
print("lazysummer_logs_query done")

# Process the data
lazysummer_logs_query.columns = lazysummer_logs_query.columns.str.lower()

# Filter deposit/staked events for relevant contracts
deposit_staked_df = lazysummer_logs_query[(lazysummer_logs_query['event_name'] == 'Deposit') | (lazysummer_logs_query['event_name'] == 'Staked')]
deposit_staked_df = deposit_staked_df[deposit_staked_df['contract_address'].isin([
    '0xa3931d71877c0e7a3148cb7eb4463524fec27fbd',
    '0x0650caf159c5a49f711e8169d4336ecb9b950275',
    '0x10ab606b067c9c461d8893c47c7512472e19e2ce'
])]

# Create synthetic referrals for LazySummer (referral code 1016)
synthetic_referals_lazysummer = deposit_staked_df.copy()[0:0]
for index, row in deposit_staked_df.iterrows():
    if row['event_name'] == 'Deposit':
        referral = row.copy()
        amount = json.loads(row['decoded_log'])['shares']
        user = json.loads(row['decoded_log'])['owner']
        referral['event_name'] = 'Referral'
        referral['decoded_log'] = '{"amount": "' + amount + '", "referral": 1016, "user": "' + user + '"}'
        synthetic_referals_lazysummer = synthetic_referals_lazysummer._append(referral)
    if row['event_name'] == 'Staked':
        staked = row.copy()
        amount = json.loads(row['decoded_log'])['amount']
        user = json.loads(row['decoded_log'])['user']
        referral['decoded_log'] = '{"amount": "' + amount + '", "referral": 1016, "user": "' + user + '"}'
        synthetic_referals_lazysummer = synthetic_referals_lazysummer._append(staked)

# Process the synthetic LazySummer data
synthetic_referals_lazysummer['dl'] = synthetic_referals_lazysummer['decoded_log'].apply(lambda x: json.loads(x))
synthetic_referals_lazysummer = synthetic_referals_lazysummer[[
    'block_number','block_timestamp', 'tx_hash',
    'event_name', 'dl','contract_address', 'contract_name','event_index', 'origin_function_signature', 'origin_from_address',
    'origin_to_address', 'topics'
]]
synthetic_referals_lazysummer['event_name'] = synthetic_referals_lazysummer['event_name'].str.lower()

# Combine all synthetic referrals
synthetic_referals = pd.concat([synthetic_referals, synthetic_referals_lazysummer])

# Display statistics about the synthetic data
referral_codes = synthetic_referals['dl'].apply(lambda x: x['referral'] if isinstance(x,dict) and 'referral' in x else 'broken').value_counts()
print(f"Synthetic LazySummer data loaded: {time.time()-timer:.2f} seconds, {synthetic_referals.shape[0]} rows")
display(referral_codes)

## 7. Data Gathering - SkyFarm Logs

Finally, we fetch events from the SkyFarm contracts.

In [ ]:
timer = time.time()

# Define the contract addresses we're interested in
contracts = [
    '0x0650caf159c5a49f711e8169d4336ecb9b950275',  # Sky Farm
    '0x10ab606B067C9C461d8893c47C7512472E19e2Ce',  # Chronicle 
    '0xa3931d71877C0E7a3148CB7Eb4463524FEc27fbD'   # sUSDS Farm
]
print("Analyzing contracts:", contracts)

# Ensure all contract addresses are lowercase
contracts = [x.lower() for x in contracts]

# Query to fetch all events from the specified contracts
sky_farm_logs_all = """
    SELECT * FROM ethereum.core.ez_decoded_event_logs WHERE contract_address IN (%s)
""" % ",".join(["('" + x.lower() + "')" for x in contracts])
dfSFLogs = pd.read_sql(sky_farm_logs_all, snowflake_conn)

# Process the data
dfSFLogs.columns = dfSFLogs.columns.str.lower()
dfSFLogs['dl'] = dfSFLogs['decoded_log'].apply(lambda x: json.loads(x))
dfSFLogs = dfSFLogs[[
    'block_timestamp', 'block_number', 'tx_hash',
    'event_name', 'dl', 'contract_address', 'contract_name', 'event_index', 'origin_function_signature', 'origin_from_address',
    'origin_to_address', 'topics'
]]
dfSFLogs['event_name'] = dfSFLogs['event_name'].str.lower()

# Get deposits with referral codes
dfDeposits = dfSFLogs[(dfSFLogs['event_name'] == 'referral') & (dfSFLogs['contract_address'] == contracts[-1])]

print(f"SkyFarm data loaded: {time.time()-timer:.2f} seconds, {dfSFLogs.shape[0]} rows")

## 8. Data Processing and Consolidation

Now we process and combine all the data for further analysis.

In [88]:
timer = time.time()

# Ensure consistent event name format
dfSFLogs['event_name'] = dfSFLogs['event_name'].str.lower()
dfSUsdsLogs['event_name'] = dfSUsdsLogs['event_name'].str.lower()

# Fill missing timestamps through interpolation
na_count = dfSFLogs['block_timestamp'].isna().sum()
dfSFLogs = dfSFLogs.sort_values(by=['block_number','event_index'], ascending=False)
dfSFLogs['block_timestamp'] = dfSFLogs['block_timestamp'].interpolate()
print(f"Filled NAs: {na_count} -> {dfSFLogs['block_timestamp'].isna().sum()}")

# Extract transfer events from sUSDS logs
dfSUsdsLogsT = dfSUsdsLogs[dfSUsdsLogs['event_name'] == 'transfer']

# Process deposit/withdraw events from sUSDS
dfUSDSB = dfSFLogs[dfSFLogs['event_name'].isin(['withdrawn','deposit','withdraw']) & 
                   (dfSFLogs['contract_address'].str.lower() == contracts[-1].lower())]
dfUSDSB['am'] = dfUSDSB.apply(
    lambda x: int(x['dl']['shares']) if x['event_name'] == 'deposit' else -1*int(x['dl']['shares']), 
    axis=1
)

# Process stake/withdraw events from SkyFarm
dfSFStakeWithdraw = dfSFLogs[dfSFLogs['event_name'].isin(['withdrawn','staked']) & 
                             (dfSFLogs['contract_address'].str.lower() == contracts[0].lower())]
dfSFStakeWithdraw['am'] = dfSFStakeWithdraw.apply(
    lambda x: int(x['dl']['amount']) if x['event_name'] == 'staked' else -1*int(x['dl']['amount']), 
    axis=1
)

# Combine all relevant events
dfRelEventsRef = dfSFLogs[dfSFLogs['event_name'].isin(['referral','staked','transfer','deposit','withdrawn','withdraw'])]
##TEST###
#shorten events
dfRelEventsRef.sort_values(['block_timestamp','event_index'], inplace=True, ascending=True)
dfRelEventsRef= dfRelEventsRef.iloc[:5000,:]

dfTraceEvents = pd.concat([dfRelEventsRef, synthetic_referals[synthetic_referals['block_number']<dfRelEventsRef['block_number'].max()]])
dfTraceEvents.sort_values(['block_timestamp','event_index'], inplace=True, ascending=True)
dfTraceEvents.reset_index(inplace=True)

# Display event statistics
event_counts = dfTraceEvents['event_name'].value_counts()
print(f"Data loaded and processed: {time.time()-timer:.2f} seconds, {dfTraceEvents.shape[0]} rows")
display(event_counts)

Filled NAs: 0 -> 0
Data loaded and processed: 0.51 seconds, 5073 rows


/tmp/ipykernel_2669817/897973127.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfUSDSB['am'] = dfUSDSB.apply(
/tmp/ipykernel_2669817/897973127.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfSFStakeWithdraw['am'] = dfSFStakeWithdraw.apply(
/tmp/ipykernel_2669817/897973127.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

event_name
staked       1732
transfer     1272
deposit       757
withdrawn     618
referral      349
withdraw      345
Name: count, dtype: int64

In [89]:
dfTraceEvents.shape

(5073, 13)

## Summary of Data Gathering Process

The data gathering process consists of several key steps:

1. **Setup and Configuration**: We import necessary libraries and set up environment variables.

2. **Database Connections**: We connect to both Snowflake (for blockchain data) and PostgreSQL (for local storage).

3. **sUSDS Logs**: We fetch and process events from the sUSDS contract.

4. **Cowswap Logs**: We fetch Cowswap-related events and create synthetic referrals with code 1003.

5. **LazySummer Logs**: We fetch LazySummer-related events and create synthetic referrals with code 1016.

6. **SkyFarm Logs**: We fetch events from the SkyFarm/Chronicle/sUSDS Farm contracts.

7. **Data Processing**: We normalize event names, fill missing timestamps, and combine all the data for analysis.

The processed data (dfTraceEvents) contains all the events we need to track referral funds, including:
- Native referral events
- Staking/deposit events
- Withdrawal events
- Transfer events
- Synthetic referral events from integrators like Cowswap and LazySummer

This data serves as the foundation for the fund tracking algorithm that follows.

# Algorithm Code

## Setup methods for tracking

In [90]:
class PotNumHolder:
    """
    A class to track and manage pot numbers.
    
    This class maintains a counter for pot numbers that can be incremented,
    retrieved, and set. Used for assigning unique identifiers to pots in the
    referral tracking system.

    Attributes:
        potnum (int): The current pot number counter
    """
    potnum = 0

    def __init__(self):
        """Initialize a new PotNumHolder with counter set to 0."""
        self.potnum = 0

    def get_potnum(self):
        """Get the current pot number.
        
        Returns:
            int: The current pot number
        """
        return self.potnum

    def set_potnum(self, potnum):
        """Set the pot number to a specific value.
        
        Args:
            potnum (int): The value to set the pot number to
        """
        self.potnum = potnum
    
    def increment_potnum(self):
        """Increment the pot number counter by 1."""
        self.potnum += 1

In [91]:
def create_new_pot(referral_code,address,amount, tx_hash, contract,liquidity_token,previous_tx_trace=[],previous_pot_trace = [], spawn = None, pot_num:PotNumHolder=None):
    """
    Creates a new pot dictionary to track referral and liquidity information.

    Args:
        referral_code (str): The referral code associated with this pot
        address (str): The wallet address that owns this pot
        amount (int): The amount of tokens in the pot
        tx_hash (str): The transaction hash that created this pot
        contract (str): The contract address this pot is associated with
        liquidity_token (str): The liquidity token address for this pot
        previous_tx_trace (list, optional): List of previous transaction hashes. Defaults to empty list.
        previous_pot_trace (list, optional): List of previous pot traces. Defaults to empty list.
        spawn (str, optional): Parent pot identifier if this was spawned from another pot. Defaults to None.
        pot_num (PotNumHolder, optional): Object to track pot numbering. Defaults to None.

    Returns:
        dict: A new pot dictionary containing all the tracking information
    """
    pot = {
        'potnum': pot_num.get_potnum(),
        'referral_code': referral_code,
        'address': address,
        'amount': amount,
        'tx_hash': tx_hash,
        'spawn': spawn,
        'initial_amount': amount,
        'contract': contract,
        'liquidity_token': liquidity_token,
        'TXtrace': previous_tx_trace + [tx_hash],
        'POTtrace': previous_pot_trace
    }
    pot_num.increment_potnum()
    return pot

In [92]:
def find_pot(from_address, ps, code=None, liquidity_token=None, contract=None):
    """
    Find a matching pot from a list of pots based on various criteria, using FIFO ordering.
    
    This function is used to locate existing pots when processing transactions and referrals.
    It helps track liquidity positions and referral relationships by finding pots that match
    the given parameters. The function implements FIFO (First In, First Out) behavior by 
    returning the first matching pot in the list, assuming pots are stored in chronological order.

    Args:
        from_address (str): The wallet address to match against pot addresses
        ps (list): List of pot dictionaries to search through, ordered by creation time (oldest first)
        code (str, optional): Referral code to match. If None, matches any code. Defaults to None.
        liquidity_token (str, optional): Token address to match. If None, matches any token. Defaults to None.
        contract (str, optional): Contract address to match. If None, matches any contract. Defaults to None.

    Returns:
        dict: First matching pot dictionary if found (FIFO order), None otherwise
    """
    for p in ps:  # Iterates in order, implementing FIFO behavior
        if (
            p['address'] == from_address 
            and (code == None or p['referral_code'] == code) 
            and p['amount'] > 0 
            and (liquidity_token == p['liquidity_token'] or liquidity_token == None)
            and ( contract == None or contract == p['contract'] )
        ):
            return p
    return None

In [93]:
def add_log(tx,timestamp,pot_id, integrator,pool,from_address,amount_delta, action, log_list=[]):
    """
    Creates a log entry for tracking pot-related events.

    Args:
        tx (str): Transaction hash
        timestamp (datetime): Timestamp of the event
        pot_id (str): Unique identifier for the pot
        integrator (str): Name/address of the integrator
        pool (str): Pool identifier
        from_address (str): Source wallet address
        amount_delta (int): Change in amount (positive for additions, negative for removals)
        action (str): Type of action performed (e.g. 'stake', 'withdraw')

    Returns:
        dict: Log entry containing all provided information
    """
    log = {
        'tx':tx,
        'timestamp':timestamp,
        'pot_id':pot_id,
        'integrator':integrator,
        'pool':pool,
        'from_address':from_address,
        'amount_delta':amount_delta,
        'action':action
    }
    log_list.append(log)
    return log

In [94]:
def contract_to_token(contract):
    """
    Maps contract addresses to their corresponding token symbols.
    
    This function takes a contract address and returns either the token symbol
    or the contract address itself based on known mappings. It handles special cases
    for specific contract addresses and empty strings.

    Args:
        contract (str): The contract address to map to a token symbol

    Returns:
        str: Either 'sUSD' for known stablecoin contracts, the original contract address,
             or 'sUSD' for empty contract strings
    """
    if contract == '0x0650caf159c5a49f711e8169d4336ecb9b950275':
        return '0x0650caf159c5a49f711e8169d4336ecb9b950275'
    elif contract == '0xa3931d71877C0E7a3148CB7Eb4463524FEc27fbD':
        return 'sUSD'
    elif contract == '':
        slog("NO CONTRACT") 
        return 'sUSD'
    else:
        return contract

In [95]:
def checkPots(pots, cutoff):
    """
    Validates pot balances against event logs up to a cutoff timestamp.
    
    This function is used for debugging to verify that the calculated pot balances
    match the expected state based on historical events. It checks each contract's
    pot balances against the event logs to detect any discrepancies.

    Args:
        pots (list): List of pot dictionaries containing balance information
        cutoff (datetime): Timestamp to check balances up to

    Returns:
        bool: True if any errors were found, False if all balances match expected state

    Usage:
        # Example usage for debugging:
        pots = [...] # List of pot states
        cutoff = pd.Timestamp('2024-01-01')
        has_errors = checkPots(pots, cutoff)
        if has_errors:
            print("Found balance discrepancies")
    """
    dfPots = pd.DataFrame(pots)
    
    dfRelEventsRefc = dfRelEventsRef[dfRelEventsRef['block_timestamp'] <= cutoff]
    error = False
    global contracts
    for c in range(len(contracts)):
        error = checkContract(c, dfPots, dfRelEventsRefc) or error
    return error

In [96]:
def withdrawFromPot(parsedEvent:rd.ReferralTransferDecoded, currentTxHash, pots, p_contract=None, 
                    p_liquidity_token=None, change_type='undefined', potnum:PotNumHolder = None, log_list = None):
    """
    Process a withdrawal or transfer from a pot, updating balances and tracking changes.
    
    Args:
        parsedEvent: The decoded transfer event containing amount and address info
        currentTxHash: Hash of the current transaction
        pots: List of pot dictionaries to update
        p_contract: Optional contract address to filter pots
        p_liquidity_token: Optional liquidity token to filter pots 
        change_type: Type of change ('withdraw' or 'transfer')
    
    Returns:
        tuple: (end, balance_changes, transfer_changes)
            end: Boolean indicating if processing should end
            balance_changes: List of balance change records
            transfer_changes: List of transfer change records
    """
    balance_changes = []
    transfer_changes = []
    # Determine pot address based on change type
    pot_og_address = parsedEvent.to_address if change_type == 'withdraw' else parsedEvent.from_address
    end = False
    f_pot = find_pot(pot_og_address, pots, contract=p_contract, liquidity_token=p_liquidity_token)
    transferred = 0
    booking_num = 0
    refcode = None

    # Process if pot is found
    if f_pot is not None:
        contract = f_pot['contract']
        liquidity_token = f_pot['liquidity_token']
        left = parsedEvent.value
        refcode = f_pot['referral_code']
        
        # Process initial reduction from first pot
        reduce = min(left, f_pot['amount'])
        left -= reduce
        transferred += reduce
        trace = {'pot':f_pot['potnum'], 'tx':currentTxHash, 'amount':reduce, 'type':change_type}
        tx_tace = [f_pot['TXtrace']]
        f_pot['amount'] -= reduce
        add_log(currentTxHash, parsedEvent.row['block_timestamp'], f_pot['potnum'], f_pot['referral_code'],
                contract_to_token(parsedEvent.contract), parsedEvent.from_address, -1*reduce, change_type, log_list=log_list)

        # Process remaining amount across multiple pots if needed
        while left > 0 and f_pot:
            f_pot = find_pot(pot_og_address, pots, contract=p_contract, liquidity_token=p_liquidity_token)
            if f_pot is not None:
                contract = f_pot['contract']
                
                # Handle case where referral code changes
                if f_pot['referral_code'] != refcode:
                    if change_type == 'transfer':
                        # Create new pot for transfer with different referral code
                        slog('New code: Different referral code in transfer', currentTxHash)
                        p = create_new_pot(refcode, parsedEvent.to_address, transferred,
                                        parsedEvent.tx_hash, contract, liquidity_token, spawn='transfer',
                                        previous_pot_trace=trace, pot_num=potnum)
                        pots.append(p)
                        add_log(currentTxHash, parsedEvent.row['block_timestamp'], p['potnum'], refcode,
                                contract_to_token(parsedEvent.contract), parsedEvent.from_address, parsedEvent.value, 'transfer',log_list=log_list)
                        transfer_change = {'ref_code':str(refcode),
                                        'amount':transferred,
                                        'timestamp':parsedEvent.row['block_timestamp'],
                                        'block_number':parsedEvent.row['block_number'],
                                        'event_index':parsedEvent.event_index,
                                        'from':parsedEvent.from_address,
                                        'to':parsedEvent.to_address,
                                        'type':'transfer',
                                        'booking_num':booking_num,
                                        'contract':parsedEvent.contract, 'tx':currentTxHash}
                        transfer_changes.append(transfer_change)
                        booking_num += 1
                        transferred = 0
                        trace = []
                        tx_tace = []
                        refcode = f_pot['referral_code']
                    else:
                        # Record balance change for withdrawal with different referral code
                        balance_change = {'ref_code':str(refcode), 'amount':transferred * -1,
                                        'timestamp':parsedEvent.row['block_timestamp'],
                                        'block_number':parsedEvent.row['block_number'],
                                        'event_index':parsedEvent.event_index,
                                        'from':parsedEvent.from_address,
                                        'type':change_type,
                                        'booking_num':booking_num,
                                        'contract':parsedEvent.contract, 'tx':currentTxHash}
                        balance_changes.append(balance_change)
                        booking_num += 1
                        transferred = 0
                        refcode = f_pot['referral_code']

                # Process reduction from current pot
                reduce = min(left, f_pot['amount'])
                left -= reduce
                add_log(currentTxHash, parsedEvent.row['block_timestamp'], f_pot['potnum'], f_pot['referral_code'],
                        contract_to_token(parsedEvent.contract), parsedEvent.from_address, -1*reduce, change_type,log_list=log_list)
                transferred += reduce
                f_pot['amount'] -= reduce
                trace = {'pot':f_pot['potnum'], 'tx':currentTxHash, 'amount':reduce, 'type':change_type}
                tx_tace.append(f_pot['TXtrace'])
            else:
                # Handle insufficient funds error
                slog('ERROR: Not enough funds in pot withdraw', currentTxHash, 'left', left/1e18, 'withdraw from', pot_og_address)
                if left/1e18 > 1:
                    end = True
                else:
                    slog('ERROR too small, ignored')
                break

        # Final processing based on change type
        if change_type == 'transfer':
            if left > 0:
                slog("ERROR left over in transfer", currentTxHash, left)
            p = create_new_pot(refcode, parsedEvent.to_address, transferred, parsedEvent.tx_hash,
                            parsedEvent.contract, liquidity_token, spawn='transfer',
                            previous_pot_trace=trace, pot_num=potnum)
            pots.append(p)
            add_log(currentTxHash, parsedEvent.row['block_timestamp'], p['potnum'], refcode,
                    contract_to_token(parsedEvent.contract), parsedEvent.from_address, parsedEvent.value, 'transfer',log_list=log_list)
            transfer_change = {'ref_code':str(refcode),
                            'amount':transferred,
                            'type':'transfer',
                            'timestamp':parsedEvent.row['block_timestamp'],
                            'block_number':parsedEvent.row['block_number'],
                            'event_index':parsedEvent.event_index,
                            'from':parsedEvent.from_address,
                            'to':parsedEvent.to_address,
                            'booking_num':booking_num,
                            'contract':parsedEvent.contract, 'tx':currentTxHash}
            transfer_changes.append(transfer_change)
            booking_num += 1
        else:
            # Record final balance change for withdrawal
            balance_change = {'ref_code':str(refcode),
                            'amount':transferred * -1,
                            'timestamp':parsedEvent.row['block_timestamp'],
                            'contract':parsedEvent.contract,
                            'block_number':parsedEvent.row['block_number'],
                            'event_index':parsedEvent.event_index,
                            'type':change_type,
                            'from':parsedEvent.from_address,
                            'booking_num':booking_num,
                            'tx':currentTxHash}
            balance_changes.append(balance_change)
            booking_num += 1
            transferred = 0
            if left > 0:
                slog("ERROR left over in withdraw", currentTxHash, left)
    else:
        # Handle case where no pot is found
        slog('ERROR: No pot found in ', change_type, currentTxHash, parsedEvent)
        if parsedEvent.value/1e18 > 1:
            end = True
        else:
            slog('ERROR too small, ignored', parsedEvent.value/1e18)

    return end, balance_changes, transfer_changes

## Algorithm implementation

In [97]:
def main(dfTraceEvents, contracts, dfsUSDSPrices, reward_percentage, slog=print):    
    dfUSDSB =  dfSFLogs[dfSFLogs['event_name'].isin(['withdrawn','deposit','withdraw']) & (dfSFLogs['contract_address'].str.lower() == contracts[-1].lower())]
    # generate sanity-check event relative to stake or withdraw
    dfUSDSB['am'] = dfUSDSB.apply(lambda x: int(x['dl']['shares'])if x['event_name'] == 'deposit' else -1*int(x['dl']['shares']), axis=1)
    dfSFStakeWithdraw = dfSFLogs[dfSFLogs['event_name'].isin(['withdrawn','staked']) & (dfSFLogs['contract_address'].str.lower() == contracts[0].lower())]

    dfSFStakeWithdraw['am'] = dfSFStakeWithdraw.apply(lambda x: int(x['dl']['amount'])if x['event_name'] == 'staked' else -1*int(x['dl']['amount']), axis=1)

    potnum = PotNumHolder()
    idxcnt = 0
    pots = []
    gstaked = {c:0 for c in contracts}
    potsOverTime = {}
    log_list = []

    ##generate empty pot and start and end of month checkpoints
    # Get all unique referral codes from events
    all_refs = dfTraceEvents['dl'].apply(lambda x: str(x.get('referral', 'untagged'))).dropna().astype(str).unique().tolist()
    all_refs.sort()

    slog("USING REFS:",all_refs, "of", dfTraceEvents.shape)

    import itertools

    # Helper function to create empty checkpoints for all contract/ref code combinations
    # These checkpoints help track balances at specific timestamps
    def empty_for_all(timestamp):
        ret = []
        for perm in itertools.product(contracts,all_refs):
            ret.append( {'ref_code': str(perm[1]),
            'amount': 0,
            'timestamp': timestamp,
            'contract': perm[0],
            'type':'checkpoint',
            'tx': f'virtual_{timestamp}_{perm[0]}_{perm[1]}'})
        return ret

    # Initialize tracking variables
    timer = time.time()
    end = False
    currentMonth = dfTraceEvents.iloc[0]['block_timestamp'].to_period('M')
    lastTime = dfTraceEvents.iloc[0]['block_timestamp']
    lastHash = None
    balance_history = []  # Tracks all balance changes
    transfer_history = [] # Tracks token transfers between addresses
    balance_history.extend(empty_for_all(currentMonth.to_timestamp()))

    # Main event processing loop
    while dfTraceEvents.shape[0] > 0 and not end:
        # Handle month transitions
        # At each month boundary, we create checkpoints to track historical balances
        nextMonth = dfTraceEvents.iloc[0]['block_timestamp'].to_period('M')
        if nextMonth != currentMonth:
            # Add checkpoints at month boundaries
            balance_history.extend(empty_for_all(nextMonth.to_timestamp()-pd.Timedelta('1s')))
            potsOverTime[currentMonth.strftime('%Y-%m')] = [copy.deepcopy(p) for p in pots]
            balance_history.extend(empty_for_all(nextMonth.to_timestamp()))
            slog('MONTH SAVED', currentMonth, '->', nextMonth, 'Augmented Timelines', nextMonth.to_timestamp()-pd.Timedelta('1s'),nextMonth.to_timestamp())
            currentMonth = nextMonth

        # Process events in current transaction
        currentTxHash = dfTraceEvents.iloc[0]['tx_hash']
        currentTxEvents = dfTraceEvents[dfTraceEvents['tx_hash'] == currentTxHash]

        # Validate pot state to ensure consistency
        if currentTxEvents['block_timestamp'].max() > lastTime and checkFrom < lastTime:
            check = checkPots(pots,lastTime)
            if check:
                slog('ERROR: Pots do not match events', 'last tx', lastHash, 'current tx', currentTxHash)
                break

        # Initialize event processing for this transaction
        to_process_events:list[rd.ReferralTransferDecoded] = []
        idxcnt +=currentTxEvents.shape[0]
        
        if idxcnt % 1000 == 0:
            slog(idxcnt)
        
        # Process stake/deposit events
        # These events represent users staking tokens or depositing into the protocol
        if 'staked' in currentTxEvents['event_name'].values or 'deposit' in currentTxEvents['event_name'].values:
            eventCandidate = currentTxEvents[currentTxEvents['event_name'].str.lower().isin(['staked','deposit'])]
            if eventCandidate.shape[0] == 1:
                to_process_events.append(rd.ReferralTransferDecoded(eventCandidate.iloc[0]))
            elif eventCandidate.shape[0] > 1:
                to_process_events = to_process_events + [rd.ReferralTransferDecoded(r[1]) for r in eventCandidate.iterrows()]
            else:
                slog('No staked events in current tx', currentTxHash)

        # Process transfer events
        # These represent token transfers between addresses, excluding mints and burns
        if 'transfer' in currentTxEvents['event_name'].values:
            eventCandidate = currentTxEvents[currentTxEvents['event_name'].str.lower().isin(['transfer'])]
            refs = [rd.ReferralTransferDecoded(r[1]) for r in eventCandidate.iterrows()]
            # Filter out mint/burn operations as they're handled differently
            to_process_events = to_process_events + [r for r in refs if not r.mint and not r.burn]

        # Process referral events
        # These events track referral relationships and rewards
        if 'referral' in currentTxEvents['event_name'].values:
            eventCandidate = currentTxEvents[currentTxEvents['event_name'].str.lower().isin([ 'referral'])]
            refs = []
            if eventCandidate.shape[0] == 1:
                refs.append(rd.ReferralTransferDecoded(eventCandidate.iloc[0]))
            elif eventCandidate.shape[0] > 1:
                for r in eventCandidate.iterrows():
                    refs.append(rd.ReferralTransferDecoded(r[1]))
            else:
                slog('No Farm events in current tx', currentTxHash)
            
            for r in refs:
                # Remove corresponding stake events to avoid double counting
                # This is necessary because referral events often come with associated stake events
                to_process_events = [x for x in to_process_events if not( 
                x.action == rd.ActionType.stake and x.from_address == r.from_address and x.value == r.value)]
                to_process_events.append(r)

        # Process withdrawal events
        # These represent users withdrawing their tokens from the protocol
        if 'withdrawn' in currentTxEvents['event_name'].values or  'withdraw' in currentTxEvents['event_name'].values:
            eventCandidate = currentTxEvents[currentTxEvents['event_name'].str.lower().isin(['withdrawn','withdraw'])]
            for r in eventCandidate.iterrows():
                to_process_events.append(rd.ReferralTransferDecoded(r[1]))

        # Sort and process all events by their index to maintain correct order
        to_process_events = sorted(to_process_events, key=lambda x: x.event_index)
        for e in to_process_events:
            parsedEvent = e
            
            # Validate referral code exists in our known set
            if parsedEvent.code is not None and str(parsedEvent.code) not in all_refs:
                slog('!!ERROR!!: New UNKNOWN referral code in tx', currentTxHash,parsedEvent.code)

            # Handle referral events
            # These create new pots for tracking referral rewards
            if parsedEvent.action == rd.ActionType.referral:
                if parsedEvent.code is not None:
                    # Update global staked amount for this contract
                    gstaked[parsedEvent.contract] += parsedEvent.value
                    # Create new pot for referral tracking
                    p=create_new_pot(
                    parsedEvent.code,
                    parsedEvent.from_address,
                    parsedEvent.value, 
                    parsedEvent.tx_hash, 
                    parsedEvent.contract,
                    contract_to_token(parsedEvent.contract),
                    previous_pot_trace=[{'pot':'spawn', 'tx':currentTxHash, 'amount':parsedEvent.value}],
                        spawn = 'ref', pot_num=potnum)
                    # Record balance change
                    balance_change = {'ref_code':parsedEvent.code, 
                                    'amount':parsedEvent.value, 
                                    'timestamp':parsedEvent.row['block_timestamp'],
                                    'block_number':parsedEvent.row['block_number'],
                                    'event_index':parsedEvent.event_index,
                                    'to':parsedEvent.from_address,
                                    'type':'referral',
                                    'contract':parsedEvent.contract, 
                                    'booking_num':0,
                                    'tx':currentTxHash}
                    balance_history.append(balance_change)
                    pots.append(p)
                    add_log(currentTxHash, parsedEvent.row['block_timestamp'], p['potnum'], parsedEvent.code, contract_to_token(parsedEvent.contract), parsedEvent.from_address, parsedEvent.value, 'referral',log_list=log_list)
                else:
                    slog('ERROR: No referral code in tx', currentTxHash)

            # Handle withdrawal events
            # These reduce pot balances and record withdrawals
            elif parsedEvent.action == rd.ActionType.withdraw:
                    f_pot = find_pot(parsedEvent.to_address, pots, contract=parsedEvent.contract)
                    tend, balance_changes, transfer_changes = withdrawFromPot(parsedEvent, currentTxHash, pots,p_contract=parsedEvent.contract, change_type='withdraw', potnum=potnum,log_list=log_list)
                    balance_history.extend(balance_changes)
                    transfer_history.extend(transfer_changes)
                    end = end or tend

            # Handle stake events
            # These create new pots for tracking staked tokens
            elif parsedEvent.action == rd.ActionType.stake:
                gstaked[parsedEvent.contract] += parsedEvent.value
                p = create_new_pot(
                    parsedEvent.code,
                    parsedEvent.from_address,
                    parsedEvent.value, 
                    parsedEvent.tx_hash, 
                    parsedEvent.contract, 
                    contract_to_token(parsedEvent.contract),
                    previous_pot_trace=[{'pot':'spawn', 'tx':currentTxHash, 'amount':parsedEvent.value}],
                    spawn='stake', pot_num=potnum)
                balance_change = {'ref_code':parsedEvent.code, 
                                'amount':parsedEvent.value, 
                                'timestamp':parsedEvent.row['block_timestamp'],
                                'block_number':parsedEvent.row['block_number'],
                                'event_index':parsedEvent.event_index,
                                'from':parsedEvent.from_address,
                                'type':'stake',
                                'booking_num':0,
                                'contract':parsedEvent.contract, 'tx':currentTxHash}
                balance_history.append(balance_change)
                pots.append(p)
                add_log(currentTxHash, parsedEvent.row['block_timestamp'], p['potnum'], parsedEvent.code, 
                        contract_to_token(parsedEvent.contract), parsedEvent.from_address, parsedEvent.value, 'stake')

            # Handle transfer events
            # These track token movements between addresses
            elif parsedEvent.type == rd.TransactionType.transfer:
                liquidity_token = parsedEvent.contract
                tend, _, transfer_changes = withdrawFromPot(parsedEvent, currentTxHash,pots, p_liquidity_token=liquidity_token, change_type='transfer',potnum=potnum, log_list=log_list)
                end = end or tend
                transfer_history.extend(transfer_changes)

        # Update tracking state for next iteration
        dfTraceEvents = dfTraceEvents[dfTraceEvents['tx_hash']!=(currentTxHash)]
        lastTime = currentTxEvents['block_timestamp'].max()
        lastHash = currentTxEvents['tx_hash'].max()

    # Handle errors and finalize
    if end:
        slog('ERROR: Pots do not match events', 'last tx', lastHash, 'current tx', currentTxHash)
        slog('shape:',dfTraceEvents.shape)
        exit(1)
    
    # Save final state for current month
    potsOverTime[currentMonth.strftime('%Y-%m')] = [copy.deepcopy(p) for p in pots]
    slog('FUNDS TRACKED', time.time()-timer)

    return pots, potsOverTime, balance_history, transfer_history

# Execute Algorithm

In [98]:
pots, potsOverTime, balance_history, transfer_history = main(dfTraceEvents, contracts, dfsUSDSPrices, reward_percentage)

/tmp/ipykernel_2669817/889053538.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfUSDSB['am'] = dfUSDSB.apply(lambda x: int(x['dl']['shares'])if x['event_name'] == 'deposit' else -1*int(x['dl']['shares']), axis=1)
/tmp/ipykernel_2669817/889053538.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfSFStakeWithdraw['am'] = dfSFStakeWithdraw.apply(lambda x: int(x['dl']['amount'])if x['event_name'] == 'staked' else -1*int(x['dl']['amount']), axis=1)


USING REFS: ['0', '1001', '1002', '1003', '1004', '1007', '13425', 'untagged'] of (5073, 13)
1000
2000
MONTH SAVED 2024-09 -> 2024-10 Augmented Timelines 2024-09-30 23:59:59 2024-10-01 00:00:00
New code: Different referral code in transfer 0x0ed2a0fd0ee2f561490bf363db382e2da0aeaa2f5b02b3908ddb944070ba16e9
New code: Different referral code in transfer 0xb376975b8c9cf64c8946d8848db86292f862aaf2488c9b826d474a3238fdd514
New code: Different referral code in transfer 0xfd7db0b7a5060d622e3f20901f62a5df793c5b5b835de954f651e3c7f699fd37
4000
5000
FUNDS TRACKED 9.725681066513062


In [99]:
dfTraceEvents

,index,block_timestamp,block_number,tx_hash,event_name,dl,contract_address,contract_name,event_index,origin_function_signature,origin_from_address,origin_to_address,topics
0,77132,2024-09-17 12:02:23,20770201,0x2235921bb1db4353575e409d36d61aeffc9cca812582...,staked,"{'amount': '442208517357432838538428', 'user':...",0x0650caf159c5a49f711e8169d4336ecb9b950275,None,466,0xa694fc3a,0x7052d6e780435e09901bd73ce280183f40a65459,0x0650caf159c5a49f711e8169d4336ecb9b950275,"[\n ""0x9e71bc8eea02a63969f509818f2dafb9254532..."
1,46740,2024-09-17 12:23:23,20770306,0x530da2be63244d31dd52590a49575d23fb3255803944...,staked,"{'amount': '7500000000000000000000', 'user': '...",0x0650caf159c5a49f711e8169d4336ecb9b950275,None,208,0x6a761202,0xb8ab6829ed0e1484050edfdfec93fceac9cac54b,0xbfc30a7b66d913c75c63ddcaaaa12e55821fc12d,"[\n ""0x9e71bc8eea02a63969f509818f2dafb9254532..."
2,46754,2024-09-17 12:36:47,20770372,0xae678e9069e0bfe775d1691267e6069c7918d596bab1...,staked,"{'amount': '141738000000000000000000', 'user':...",0x0650caf159c5a49f711e8169d4336ecb9b950275,None,107,0xa694fc3a,0x344d9c4f488bb5519d390304457d64034618145c,0x0650caf159c5a49f711e8169d4336ecb9b950275,"[\n ""0x9e71bc8eea02a63969f509818f2dafb9254532..."
3,46796,2024-09-17 12:42:59,20770403,0x503d941e3efb4e496da6eaf6f64336f6a9c1065c968a...,staked,"{'amount': '400000000000000000000000', 'user':...",0x0650caf159c5a49f711e8169d4336ecb9b950275,None,265,0xa694fc3a,0x344d9c4f488bb5519d390304457d64034618145c,0x0650caf159c5a49f711e8169d4336ecb9b950275,"[\n ""0x9e71bc8eea02a63969f509818f2dafb9254532..."
4,45211,2024-09-17 12:47:59,20770428,0x43df1618391794cb266020f80a9bb842de49a8f2d605...,staked,"{'amount': '1000000000000000000000', 'user': '...",0x10ab606b067c9c461d8893c47c7512472e19e2ce,None,356,0xa694fc3a,0x4178824eac1d03445477772e71f4a53a5ee53184,0x10ab606b067c9c461d8893c47c7512472e19e2ce,"[\n ""0x9e71bc8eea02a63969f509818f2dafb9254532..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5068,46962,2024-10-14 21:22:47,20966461,0x2d3cebb83046389675df27af965ec82c2fc3085dc3c9...,withdraw,"{'assets': '873997403724916627273173', 'owner'...",0xa3931d71877c0e7a3148cb7eb4463524fec27fbd,Savings USDS,529,0x22bee494,0x0a4d75ab96375e37211cd00a842d77d0519eed1b,0x604e586f17ce106b64185a7a0d2c1da5bace711e,"[\n ""0xfbde797d201c681b91056529119e0b02407c7b..."
5069,34934,2024-10-14 21:40:59,20966552,0xfce92c5fcbc154d4299fdd40536c683e050cfc905aa3...,deposit,"{'assets': '624493202285950034386372', 'owner'...",0xa3931d71877c0e7a3148cb7eb4463524fec27fbd,Savings USDS,205,0x6a761202,0xea1b76f2cd671cfe29893b4ac778d22511058048,0x485947c54ac21c6317559c88440d7d2394352716,"[\n ""0xdcbc1c05240f31ff3ad067ef1ee35ce4997762..."
5070,86728,2024-10-14 21:40:59,20966552,0xfce92c5fcbc154d4299fdd40536c683e050cfc905aa3...,transfer,{'from': '0x0000000000000000000000000000000000...,0xa3931d71877c0e7a3148cb7eb4463524fec27fbd,Savings USDS,206,0x6a761202,0xea1b76f2cd671cfe29893b4ac778d22511058048,0x485947c54ac21c6317559c88440d7d2394352716,"[\n ""0xddf252ad1be2c89b69c2b068fc378daa952ba7..."
5071,58353,2024-10-14 22:11:35,20966704,0xe85726021ce0d69a00a5450bb0fe4ed757866377c3f2...,withdrawn,"{'amount': '819028640110005629734981', 'user':...",0x0650caf159c5a49f711e8169d4336ecb9b950275,None,399,0x22bee494,0x0a4d75ab96375e37211cd00a842d77d0519eed1b,0x604e586f17ce106b64185a7a0d2c1da5bace711e,"[\n ""0x7084f5476618d8e60b11ef0d7d3f06914655ad..."


# check for correctness

In [100]:
len(pots)

2663

In [ ]:
# %% 
# Initialize logging and prepare balance history dataframe
slog = print
dfBH = pd.DataFrame(balance_history)
dfBH['ref_code'] = dfBH['ref_code'].astype(str)
code_contract_uniques = dfBH[['contract','ref_code']].value_counts().reset_index()

# Constants for reward calculations
total_seconds = 60 * 60 * 24 * 365
slog("using reward %:",reward_percentage, "on total seconds:", total_seconds)

# Helper function to convert TVL to USD value for sUSDS token
def rowToUSDValue(row):
    #safety check to prevent accidental wrong usage,
    # hence DF requires the contract field as well
    if row['contract'] == '0xa3931d71877c0e7a3148cb7eb4463524fec27fbd'.lower():
        month_price = dfsUSDSPrices[dfsUSDSPrices['month'] == row['month']]['price']
        return row['eligible_tvl'] * month_price.values[0] if month_price.shape[0]>0 else row['eligible_tvl']
    else:
        print("wrong address called")
        return row['eligible_tvl']

# %%


using reward %: 0.004 on total seconds: 31536000
___________ 0x0650caf159c5a49f711e8169d4336ecb9b950275 untagged 0 ___________
___________ 0xa3931d71877c0e7a3148cb7eb4463524fec27fbd untagged 1 ___________
___________ 0x10ab606b067c9c461d8893c47c7512472e19e2ce untagged 2 ___________
___________ 0xa3931d71877c0e7a3148cb7eb4463524fec27fbd 0 3 ___________
___________ 0xa3931d71877c0e7a3148cb7eb4463524fec27fbd 1003 4 ___________
___________ 0x0650caf159c5a49f711e8169d4336ecb9b950275 1001 5 ___________
___________ 0xa3931d71877c0e7a3148cb7eb4463524fec27fbd 1001 6 ___________
___________ 0x10ab606b067c9c461d8893c47c7512472e19e2ce 1001 7 ___________
___________ 0x0650caf159c5a49f711e8169d4336ecb9b950275 1007 8 ___________
___________ 0x0650caf159c5a49f711e8169d4336ecb9b950275 13425 9 ___________
___________ 0xa3931d71877c0e7a3148cb7eb4463524fec27fbd 1002 10 ___________
___________ 0xa3931d71877c0e7a3148cb7eb4463524fec27fbd 13425 11 ___________
___________ 0xa3931d71877c0e7a3148cb7eb4463524fec2

In [107]:
60*60*24 * 30

2592000


Next we generate cumsum and weighted TVL seconds from the given balance history (BH) data
## Mathematical Explanation

For each referral code and contract pair, we calculate:

### 1. Cumulative Balance ($B_t$)
$B_t = \sum_{i=0}^t amount_i$

Where $amount_i$ represents individual deposits/withdrawals

### 2. Time Weight ($W_t$)
The time weight factor ensures rewards are proportional to how long funds are held:

$W_t = \frac{t - t_{prev}}{seconds_{year}}$

Where:
- $t$ is the current timestamp
- $t_{prev}$ is the previous timestamp 
- $seconds_{year}$ normalizes to annual basis (31,536,000 seconds)

This creates proportional rewards by:
1. Shorter holding periods result in smaller weights (e.g. 1 day = ~0.003)
2. Longer holding periods have larger weights (e.g. 6 months = ~0.5) 
3. Annual normalization keeps rewards consistent regardless of time period

For example:
- $100 held for 1 day: $100 * (86400(60* 60 * 24)/31,536,000) = ~$0.27 weighted TVL
- $100 held for 1 month: $100 * (2592000 (60* 60 * 24 * 30)/31,536,000) = ~$8.22 weighted TVL

Where $t_{prev}$ is the previous timestamp

### 3. TVL-seconds ($TVL_t$)
$TVL_t = B_t \times W_t$

This weights the balance by duration held

### 4. Monthly Eligible TVL
$TVL_{month} = \sum_{t \in month} TVL_t$

Final TVL is in USD terms (special conversion for sUSDS token)

### Implementation Steps
1. Group data by referral code & contract
2. Sort by timestamp to calculate proper cumulative sums
3. Calculate running balance using `cumsum()`
4. Weight by held time (time diff between entries)
5. Multiply balance × time weight for TVL-seconds
6. Group by month and sum for monthly rewards
7. Convert sUSDS token amounts to USD value
8. Aggregate across all codes/contracts


In [ ]:
# Calculate rewards for each referral code and contract combination
dfBHs = []
dfBHsR = []
dfRewards = None
i = 0
for r in code_contract_uniques.iterrows():
    # Get data for current contract/ref code pair
    current_contract = r[1]['contract']
    dfBHt = dfBH[(dfBH['contract'] == current_contract) & (dfBH['ref_code'] == r[1]['ref_code'])].copy()
    dfBHt.sort_values('timestamp',inplace=True)
    dfBHsR.append(dfBHt)
    
    # Calculate cumulative sums and TVL
    dfBHt.loc[:,'cumsum'] = dfBHt['amount'].cumsum()
    dfBHt.loc[:,'held_time'] = dfBHt['timestamp'].diff().replace(pd.NaT,pd.Timedelta(seconds=1)).dt.total_seconds()/total_seconds
    dfBHt.loc[:,'eligible_tvl'] = dfBHt['cumsum'] * dfBHt['held_time']
    dfBHt.loc[:,'month'] = dfBHt['timestamp'].dt.to_period('M')
    
    # Special handling for sUSDS token to convert to USD value
    if current_contract == '0xa3931d71877c0e7a3148cb7eb4463524fec27fbd':
        testTail = dfBHt['eligible_tvl'].tail().copy()
        dfBHt.loc[:,'eligible_tvl'] = dfBHt.apply(lambda x: rowToUSDValue(x), axis=1)

    # Aggregate rewards by month
    dfReward = (dfBHt.groupby('month')['eligible_tvl'].sum()/1e18).reset_index()
    dfReward.loc[:,'contract'] = current_contract
    dfReward.loc[:,'ref_code'] = r[1]['ref_code']
    
    # Combine with overall rewards
    if dfRewards is None:
        dfRewards = dfReward
    else:
        dfRewards = pd.concat([dfRewards,dfReward])
    dfBHs.append(dfBHt)
    slog('___________', current_contract, r[1]['ref_code'], i,'___________')
    i+=1

# Final reward calculations and cleanup    
dfRewards.rename(columns={'ref_code':'referral_code'},inplace=True)
dfRewards['eligible_tvl'] = dfRewards['eligible_tvl'].astype(np.float64)
dfRAgg = dfRewards.groupby(['month', 'referral_code'])['eligible_tvl'].sum().reset_index()

slog("Done calculating rewards", dfRAgg.index)

# Calculate final payouts
dfRAggPayout = dfRAgg
dfRAggPayout['payout'] = dfRAggPayout['eligible_tvl'] * reward_percentage
dfRAggPayout

# %%
# Pot analysis
ps = pd.DataFrame(pots)

# %%
# Calculate total amount in pots
dfPots = pd.DataFrame(pots)
dfPots['amount'].sum() / 1e18

# %%
# Helper function to calculate amount changes from events
def rowToAmount(row):
    if row['event_name'] == 'deposit':
        return int(row['dl']['shares'])
    elif row['event_name'] == 'withdraw':
        return -1*int(row['dl']['shares'])
    elif row['event_name'] == 'withdrawn':
        return -1*int(row['dl']['amount'])
    elif row['event_name'] == 'staked':
        return int(row['dl']['amount'])
    else:
        return 0

# %%
# Validation function to check contract balances
def checkContract(conNum):
    dfRelEventsRef[(dfRelEventsRef['contract_address'] ==contracts[conNum])]['event_name'].value_counts()
    stakeSum = (dfRelEventsRef[(dfRelEventsRef['event_name'] != 'referral') & (dfRelEventsRef['contract_address'] ==contracts[conNum])  ].apply(lambda x: rowToAmount(x),axis=1).sum() / 1e18)
    stakeSum
    computed_sum = dfPots[dfPots['contract'] == contracts[conNum]]['amount'].sum() / 1e18
    if abs( stakeSum - (computed_sum))>1:
        return " ".join([str(x) for x in ['\n🔴!!ERROR!!🔴\n Contract', contracts[conNum], "stake sum mismatch",
                                        stakeSum, (computed_sum),
                                        'OFF BY:', stakeSum - computed_sum,
                                        "=>", f'{ 100*(abs(stakeSum / computed_sum)-1):3.5f}%']])
    else:
        return " ".join([str(x) for x in ['✅OK!!✅ Contract', contracts[conNum], "stake sum MATCH",
                                        stakeSum, computed_sum]])

# %%
# Run validation checks
slog('\n!!SANITY CHECKS!!')
logs = []
for c in range(len(contracts)):
    logs.append(checkContract(c))
for l in logs:
    slog(l)
slog('!!SANITY CHECKS!!\n')